# 预测录取情况 
## 多层神经网络  
### 采用梯度下降.
### 6 *n*1的网络结构

In [1]:
import numpy as np
import pandas as pd

admissions = pd.read_csv(r'C:\Users\zhangwenqi\Desktop\神经网络test\第一课神经网络\第三章 实现梯度下降\binary.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(21)
sample = np.random.choice(data.index, size=int(len(data)*0.8), replace=False)
data, test_data = data.ix[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))


# Hyperparameters
n_hidden = 2  # number of hidden units  6*3*1的结构
epochs = 1000
learnrate = 0.005

n_records, n_features = features.shape
last_loss = None
# Initialize weights
weights_input_hidden = np.random.normal(scale=1 / n_features ** .5,
                                        size=(n_features, n_hidden))  #6*3矩阵 从输入到隐层
weights_hidden_output = np.random.normal(scale=1 / n_features ** .5,
                                         size=(n_hidden)) # 1维数组 向量  隐层到输出

for e in range(epochs):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape) #  6*3矩阵 
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)  #3元素向量矩阵与 权重矩阵一致
    for x, y in zip(features.values, targets): #x是6元素行向量 
        ## Forward pass ##
        # TODO: Calculate the output
        hidden_input = np.dot(x, weights_input_hidden)#向量 dot 6*3 =3元素行向量  对应三个隐层输入
        hidden_output = sigmoid(hidden_input)  

        output = sigmoid(np.dot(hidden_output,
                                weights_hidden_output)) # 向量内积为输出

        ## Backward pass ##
        # TODO: Calculate the network's prediction error
        error = y - output   #输出层误差

        # TODO: Calculate error term for the output unit
        output_error_term = error * output * (1 - output) #输出层的梯度 

        ## propagate errors to hidden layer

        # TODO: Calculate the hidden layer's contribution to the error
        hidden_error = np.dot(output_error_term, weights_hidden_output)   #隐层误差  3元素行向量 对应三个隐层节点的误差
        
        # TODO: Calculate the error term for the hidden layer
        hidden_error_term = hidden_error * hidden_output * (1 - hidden_output)  #隐层误差梯度  行向量 

        # TODO: Update the change in weights
        del_w_hidden_output += output_error_term * hidden_output  #    1*3 的输出代表隐层到输出层的权重更新值
        del_w_input_hidden += hidden_error_term * x[:, None]  # 向量3 *  矩阵6*1 =6*3矩阵

    # TODO: Update weights
    weights_input_hidden += learnrate * del_w_input_hidden / n_records 
    weights_hidden_output += learnrate * del_w_hidden_output / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output,
                             weights_hidden_output))
        loss = np.mean((out - targets) ** 2)

        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

c:\users\zhangwenqi\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


Train loss:  0.22922394667412999
Train loss:  0.22898059361787163
Train loss:  0.2287439636033973
Train loss:  0.22851386821664574
Train loss:  0.2282901242364174
Train loss:  0.22807255351449912
Train loss:  0.22786098285673262
Train loss:  0.2276552439051697
Train loss:  0.2274551730214494
Train loss:  0.22726061117151258
Prediction accuracy: 0.688


In [109]:
x

array([-1.27860482, -1.6551629 ,  0.        ,  1.        ,  0.        ,
        0.        ])

In [110]:
weights_input_hidden

array([[ 0.02532096, -0.24502934,  0.06503876],
       [ 0.3006753 ,  0.5415318 , -0.47025659],
       [-0.49588069,  0.00436551,  0.31563855],
       [-0.69171741, -0.33620111,  0.5696968 ],
       [-0.50294035, -0.16800105, -0.26333521],
       [-0.03352158,  0.47661494,  0.2123448 ]])

In [114]:
hidden_input = np.dot(x, weights_input_hidden)#1*6 dot 6*3 =1*3 为隐层


In [117]:
hidden_error_term

array([ 0.01137862, -0.00250703,  0.00396628])

In [118]:
 x[:, None]

array([[-1.27860482],
       [-1.6551629 ],
       [ 0.        ],
       [ 1.        ],
       [ 0.        ],
       [ 0.        ]])

In [120]:
hidden_error_term* x[:, None]

array([[-0.01454876,  0.00320551, -0.0050713 ],
       [-0.01883347,  0.00414955, -0.00656483],
       [ 0.        , -0.        ,  0.        ],
       [ 0.01137862, -0.00250703,  0.00396628],
       [ 0.        , -0.        ,  0.        ],
       [ 0.        , -0.        ,  0.        ]])